In [1]:
import datetime
import os
import json
import pandas as pd
from typing import Union
import numpy as np
import sys
import datetime as dt
import logging
import time

sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/trading/agency_desk/lib/")
import talos_utils

fred_api_key = '4d7dc5c95513ef9bb84078e6d0fae493'
coinglass_api = '06b33af895f04b218cce5745a71275ec'
messari_api = 'cae559e4-f675-4c26-bb8b-545ff38594ca'
talos_api = 'ANC80D4RI4DL'
talos_secret_api = 'mv1ahb471yqfij1lt9g8bamhokibonwq'

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.8f}'.format

In [2]:
talos_secret = 'mv1ahb471yqfij1lt9g8bamhokibonwq'
host = "tal-42.prod.talostrading.com"
talos = talos_utils.Talos(talos_api, talos_secret, host)

talos_adv_api = 'ANC0ASVTOGM0'
talos_adv_secret_api = '8myg2ro5bf2pzfx4m79u9yh501e09nky'
host_wl = "tal-160.prod.talostrading.com"
talos_wl = talos_utils.Talos(talos_adv_api, talos_adv_secret_api, host_wl)

In [3]:
talos_secret = 'mv1ahb471yqfij1lt9g8bamhokibonwq'
host = "tal-42.prod.talostrading.com"
talos = talos_utils.Talos(talos_api, talos_secret, host)

talos_adv_api = 'ANC0ASVTOGM0'
talos_adv_secret_api = '8myg2ro5bf2pzfx4m79u9yh501e09nky'
host_wl = "tal-160.prod.talostrading.com"
talos_wl = talos_utils.Talos(talos_adv_api, talos_adv_secret_api, host_wl)

### Calculator function, like google sheets

In [4]:
def commission_calculator(side, symbol, filled_qty, filled_px, commission, filled_ccy):
    side = side.lower()
    filled_qty = float(filled_qty)
    filled_px = float(filled_px)

    if side not in ["buy", "sell"]:
        raise ValueError("Side must be either Buy or Sell")
    
    base_asset, quote_asset = symbol.split("-")
    is_buy_order = side == "buy"

    if is_buy_order:
        buy_asset = base_asset
        sell_asset = quote_asset
    else:
        buy_asset = quote_asset
        sell_asset = base_asset

    if filled_ccy == buy_asset:
        amount = filled_qty*filled_px if is_buy_order else filled_qty/filled_px
        comms = amount*commission/10000
        amount_with_comms = amount + comms
        price_with_comms = amount_with_comms/filled_qty if is_buy_order else filled_qty/amount_with_comms
        bought_amount = filled_qty
        sold_amount = amount_with_comms
        comms_ccy = sell_asset
    elif filled_ccy == sell_asset:
        amount = filled_qty/filled_px if is_buy_order else filled_qty*filled_px
        comms = amount*commission/10000
        amount_with_comms = amount - comms
        price_with_comms = filled_qty/amount_with_comms if is_buy_order else amount_with_comms/filled_qty
        bought_amount = amount_with_comms
        sold_amount = filled_qty
        comms_ccy = buy_asset

    bought_amount = round(bought_amount, 8) if buy_asset != "USD" else round(bought_amount, 2)
    sold_amount = round(sold_amount, 8) if sell_asset != "USD" else round(sold_amount, 2)
    comms = round(comms, 8) if comms_ccy != "USD" else round(comms, 2) 
    price_with_comms = round(price_with_comms, 8)
    
    output = {
        "trade_side": side,
        "bought_amount": bought_amount,
        "sold_amount": sold_amount,
        "commission": comms,
        "talos_price": filled_px,
        "price_with_comms": price_with_comms,
        "bought_asset": buy_asset,
        "sold_asset": sell_asset,
        "comms_ccy": comms_ccy,
        "amount_gross": amount,
    }

    return output

### Calculate from talos order ids

In [5]:
def fetch_order(order_id, filled_and_cancelled_threshold:float = 0.999, client_name:str = None, order_desc:str = None, talos_env = "principal"):
    if talos_env == "principal":
        order = talos.get_orders(order_id=order_id)[0]
        time.sleep(2)
    elif talos_env == "wl":
        order = talos_wl.get_orders(order_id=order_id)[0]
        time.sleep(2)
    order["FilledPct"] = float(order["CumQty"])/float(order["OrderQty"])
    order["TradedQty"] = order["CumQty"] if order["FilledPct"] < filled_and_cancelled_threshold else order["OrderQty"]
    order["ClientName"] = client_name
    order["OrderDesc"] = order_desc
    if not order.get("Strategy"):
        order["Strategy"] = "Market Order"
    if not order.get("AvgPxAllIn"):
        order["AvgPxAllIn"] = 0
    return order

def calculate_comms(order_details: Union[str, list, dict] = None, client_name:str = "", order_desc:str = "", commission:float = 10, filled_and_cancelled_threshold:float = 0.999):
    order_talos = {}
    commissions_dict = {}
    if order_details:
        if isinstance(order_details, str):
            if len(client_name) == 0:
                raise ValueError("Client name must be provided")
            order = fetch_order(order, filled_and_cancelled_threshold, client_name, order_desc)
            time.sleep(0.5)
            order["print_output"] = True
            for key, value in order.items():
                if key in order_talos:
                    order_talos[key].append(value)
                else:
                    order_talos[key] = [value]
        if isinstance(order_details, list):
            if len(client_name) == 0:
                raise ValueError("Client name must be provided")
            orders_list = []
            for order in order_details:
                order = fetch_order(order, filled_and_cancelled_threshold, client_name, order_desc)
                time.sleep(0.5)
                order["print_output"] = True
                orders_list.append(order)
            for result in orders_list:
                for key, value in result.items():
                    if key in order_talos:
                        order_talos[key].append(value)
                    else:
                        order_talos[key] = [value]
        if isinstance(order_details, dict):
            orders_list = []
            for order in order_details.keys():
                print_output = order_details[order]["print_output"]
                order = fetch_order(order, filled_and_cancelled_threshold, order_details[order]["client_name"], order_details[order]["order_desc"], order_details[order]["talos_env"])
                time.sleep(0.5)
                order["print_output"] = print_output
                orders_list.append(order)
            for result in orders_list:
                for key, value in result.items():
                    if key in order_talos:
                        order_talos[key].append(value)
                    else:
                        order_talos[key] = [value]
            
        
    replacements = {"MarketTWAP": "TWAP", "DoneForDay": "Done", "PartiallyFilled": "Open"}
    strategies = [replacements.get(item, item) for item in order_talos["Strategy"]]
    order_status = [replacements.get(item, item) for item in order_talos["OrdStatus"]]

    order_params = {
        "Date": order_talos["SubmitTime"],
        "Side": order_talos["Side"],
        "Symbol": order_talos["Symbol"],
        "FilledQty": order_talos["TradedQty"],
        "FilledPrice": order_talos["AvgPxAllIn"],
        "FilledCcy": order_talos["Currency"],
        "FilledAmount": order_talos["AmountCurrency"],
        "OrderType": strategies,
        "OrderID": order_talos["OrderID"],
        "OrdStatus": order_status,
        "ClientName": order_talos["ClientName"],
        "OrderDesc": order_talos["OrderDesc"],
        "print_output": order_talos["print_output"]
    }

    #print(json.dumps(order_params, indent=4))

    try:
        for i in range(0,len(order_params["OrderID"])):
            if order_params["FilledQty"][i] == '0':
                continue
            
            commission_calc = commission_calculator(order_params["Side"][i], order_params["Symbol"][i], order_params["FilledQty"][i], order_params["FilledPrice"][i], commission, order_params["FilledCcy"][i])
            commission_calc["OrderType"] = order_params["OrderType"][i]
            commission_calc["OrdStatus"] = order_params["OrdStatus"][i]
            commission_calc["Symbol"] = order_params["Symbol"][i]
            commission_calc["Date"] = pd.to_datetime(order_params["Date"][i])
            commission_calc["Date"] = commission_calc["Date"].strftime("%Y-%m-%d %H:%M:%S")
            commissions_dict[order_params["OrderID"][i]] = commission_calc
            commissions_dict[order_params["OrderID"][i]]["ClientName"] = order_params["ClientName"][i]
            commissions_dict[order_params["OrderID"][i]]["OrderDesc"] = order_params["OrderDesc"][i]
            commissions_dict[order_params["OrderID"][i]]["print_output"] = order_params["print_output"][i]
    except Exception as e:
        print(f"Error in commission calculation for order {order_params['OrderID'][i]}: {e}")
        pass

    orders_df = pd.DataFrame(commissions_dict).T
    orders_df = orders_df.infer_objects()

    return orders_df[['ClientName', 'OrderDesc', 'trade_side', 'bought_amount', 'bought_asset', 'sold_amount', 'sold_asset', 'commission', 'comms_ccy', 'price_with_comms', 'talos_price', 'amount_gross', 'OrderType', 'OrdStatus', 'Symbol', 'Date', 'print_output']]

### Function to generate order update text

In [6]:
def generate_order_update_text(orders_df, show_finished_orders=False):
    df = orders_df.copy()
    df["direction"] = np.where(df["trade_side"]=="buy", "bought", "sold")
    df = df.sort_values(by=["direction", "Symbol"])
    open_orders = df[df["OrdStatus"]=="Open"]
    closed_orders = df[df["OrdStatus"]=="Done"]

    crypto_format = ",.8f"
    usd_format = ",.2f"

    order_update_text = ""
    header = f"Order Update\n\n"
    open_orders_text = "Open Orders:\n"

    for i in range (0, len(open_orders)):
        row = open_orders.iloc[i]
        if row["direction"] == "bought":
            asset = row["bought_asset"]
            notional_asset = row["sold_asset"]
        else:
            asset = row["sold_asset"]
            notional_asset = row["bought_asset"]

        client_name = row["ClientName"]
        order_description = row["OrderDesc"] if row["OrderDesc"] else None
        direction = row["direction"]
        amount = row["bought_amount"] if direction == "bought" else row["sold_amount"]
        amount_formatted = format(amount, crypto_format) if asset != "USD" else format(amount, usd_format)
        price = row["price_with_comms"]
        notional = float(amount) * float(price)
        notional_formatted = format(notional, crypto_format) if notional_asset != "USD" else f"${format(notional, usd_format)}"
        price_formatted = format(price, ",.8f") if notional_asset != "USD" else f"${format(price, ',.8f')}"    
        open_orders_text += f"\n{order_description}\n" if order_description else ""
        open_orders_text += f"{client_name} {direction} {amount_formatted} {asset} @ {price_formatted} for {notional_formatted} net\n"

    order_update_text += header + open_orders_text

    if len(closed_orders) > 0:

        all_orders_text = "\nCumulatively:"
        all_orders_grouped = df.groupby(by=["ClientName", "Symbol", "direction"], as_index=False).agg(lambda x: x.sum() if x.dtype == "float64" else ", ".join(set(", ".join(map(str, x)).replace("-", ", ").split(", "))))
        all_orders_grouped = all_orders_grouped.sort_values(by=["direction", "Symbol"])
        for i in range (0, len(all_orders_grouped)):
            row = all_orders_grouped.iloc[i]
            client_name = row["ClientName"]
            if row["direction"] == "bought":
                asset = row["bought_asset"]
                notional_asset = row["sold_asset"]
                price = row["sold_amount"]/row["bought_amount"]
            else:
                asset = row["sold_asset"]
                notional_asset = row["bought_asset"]
                price = row["bought_amount"]/row["sold_amount"]

            direction = row["direction"]    
            amount = row["bought_amount"] if direction == "bought" else row["sold_amount"]
            amount_formatted = format(amount, crypto_format) if asset != "USD" else format(amount, usd_format)
            notional = float(amount) * float(price)
            notional_formatted = format(notional, crypto_format) if notional_asset != "USD" else f"${format(notional, usd_format)}"
            price_formatted = format(price, ",.8f") if notional_asset != "USD" else f"${format(price, ',.8f')}"

            all_orders_text += f"\n{client_name} {direction} {amount_formatted} {asset} @ {price_formatted} for {notional_formatted} net"        
        
        if show_finished_orders:
            all_orders_text += "\n\nFilled Orders:\n"
            for i in range (0, len(closed_orders)):
                row = closed_orders.iloc[i]
                print_output = row["print_output"]
                if not print_output:
                    continue
                if row["direction"] == "bought":
                    asset = row["bought_asset"]
                    notional_asset = row["sold_asset"]
                else:
                    asset = row["sold_asset"]
                    notional_asset = row["bought_asset"]

                client_name = row["ClientName"]
                order_description = row["OrderDesc"] if row["OrderDesc"] else None
                direction = row["direction"]
                amount = row["bought_amount"] if direction == "bought" else row["sold_amount"]
                amount_formatted = format(amount, crypto_format) if asset != "USD" else format(amount, usd_format)
                price = row["price_with_comms"]
                notional = float(amount) * float(price)
                notional_formatted = format(notional, crypto_format) if notional_asset != "USD" else f"${format(notional, usd_format)}"
                price_formatted = format(price, ",.8f") if notional_asset != "USD" else f"${format(price, ',.8f')}"    
                all_orders_text += f"\n{order_description}\n" if order_description else ""
                all_orders_text += f"{client_name} {direction} {amount_formatted} {asset} @ {price_formatted} for {notional_formatted} net\n"


        order_update_text += all_orders_text

    return order_update_text

# Order Updates

In [7]:
###orders updates new

sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/trading/agency_desk")
from lib import talos_utils

sys.path.append(os.environ["HOME"] + "/anchorage/source/python/lib/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility # type: ignore
import utils as aql_utils # type: ignore

os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
google_sheet_key = aql_utils.read_secret(os.environ.get("GOOGLE_SHEET_KEY"))
gsheet_key = json.loads(google_sheet_key)
worksheet_name = "Order Update Script Sheet"
tab_name = "Paradigm"
gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)

df = gsu._get_current_sheet_df(tab_name, 0)

# Initialize an empty dictionary
orders_list_gs = {}

# Iterate through the DataFrame and populate the orders_list_gs dictionary
for _, row in df.iterrows():
    try:
        orders_list_gs[row['order_id']] = {
            "client_name": row['client_name'],
            "order_desc": "",  # Empty string for 'order_desc'
            "print_output": bool(row['print_output']),
            "talos_env": row['talos_env']
        }
    except Exception as e:
        print(f"Error processing order_id {row['order_id']}: {e}")

# Start the timer
startTime = time.time()

paradigm_orders_comms_df = calculate_comms(order_details=orders_list_gs, client_name="Paradigm Fund", commission = 10)
paradigm_update = generate_order_update_text(paradigm_orders_comms_df, show_finished_orders=True)
print(paradigm_update)

endTime = time.time()
elapsedTime = endTime - startTime
print("Elapsed Time = %s" % elapsedTime)

/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


KeyboardInterrupt: 

In [13]:
sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/trading/agency_desk")
from lib import talos_utils

sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility
import utils as aql_utils

os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
google_sheet_key = aql_utils.read_secret(os.environ.get("GOOGLE_SHEET_KEY"))
gsheet_key = json.loads(google_sheet_key)
worksheet_name = "Order Update Script Sheet"
tab_name = "pantera"
gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)

df = gsu._get_current_sheet_df(tab_name, 0)

# Initialize an empty dictionary
orders_list_gs = {}

# Iterate through the DataFrame and populate the orders_list_gs dictionary
for _, row in df.iterrows():
    try:
        orders_list_gs[row['order_id']] = {
            "client_name": row['client_name'],
            "order_desc": "",  # Empty string for 'order_desc'
            "print_output": bool(row['print_output']),
            "talos_env": row['talos_env']
        }
    except Exception as e:
        print(f"Error processing order_id {row['order_id']}: {e}")

# Start the timer
startTime = time.time()

paradigm_orders_comms_df = calculate_comms(order_details=orders_list_gs, client_name="Coatue Crypto", commission = 10)
paradigm_update = generate_order_update_text(paradigm_orders_comms_df, show_finished_orders=True)
print(paradigm_update)

endTime = time.time()
elapsedTime = endTime - startTime
print("Elapsed Time = %s" % elapsedTime)

/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Order Update

Open Orders:

Cumulatively:
Ribbit OB1 (Master), L.P. bought 209.82353251 BTC @ $62,178.59743343 for $13,046,532.96 net
Ribbit OB1 (Master), L.P. bought 79.81048160 BTC @ 61,688.30268925 for 4,923,373.14671589 net

Filled Orders:
Ribbit OB1 (Master), L.P. bought 209.82353251 BTC @ $62,178.59743913 for $13,046,532.96 net
Ribbit OB1 (Master), L.P. bought 79.81048160 BTC @ 61,688.30269209 for 4,923,373.14694228 net

Elapsed Time = 5.980887413024902


In [11]:
orders_list = ["720c66aa-b06e-41fe-87a5-086f4d306731"]
client_name = "Paradigm Fund"
client_orders_comms_df = calculate_comms(order_details=orders_list, client_name=client_name, commission = 40)
client_update = generate_order_update_text(client_orders_comms_df, show_finished_orders=False)
print(client_update)

IndexError: list index out of range

In [ ]:
orders_list = ["c73ce8ba-4a1f-468e-8cb1-6f7c2c5d16cb"]
client_name = "Harish Devarajan Individual Account"
client_orders_comms_df = calculate_comms(order_details=orders_list, client_name=client_name, commission = 10)
client_update = generate_order_update_text(client_orders_comms_df, show_finished_orders=False)
print(client_update)

Order Update

Open Orders:

Cumulatively:
Harish Devarajan Individual Account sold 31,010.76000000 STRK @ $1.11451606 for $34,561.99 net


In [11]:
orders_list = ["677b7153-6bf1-494e-bec0-e25874495fca","d4ccadb8-4565-4812-adf5-b2d003fc511f"]
client_name = "XXX"
client_orders_comms_df = calculate_comms(order_details=orders_list, client_name=client_name, commission = 0)
client_update = generate_order_update_text(client_orders_comms_df, show_finished_orders=False)
print(client_update)

IndexError: list index out of range

In [ ]:
orders_list = {
                 "7f9fcb2a-3683-44fe-83d2-6c6d9c5432ab" : {"client_name": "North Island Ventures", "order_desc": "", "print_output": True, "talos_env": "wl"},
}

paradigm_orders_comms_df = calculate_comms(order_details=orders_list, client_name="Paradigm Fund", commission = 10)
paradigm_update = generate_order_update_text(paradigm_orders_comms_df, show_finished_orders=True)
print(paradigm_update)

Order Update

Open Orders:

Cumulatively:
North Island Ventures bought 641.57487093 MKR @ $3,117.32907665 for $2,000,000.00 net

Filled Orders:
North Island Ventures bought 641.57487093 MKR @ $3,117.32907667 for $2,000,000.00 net



In [9]:
###orders updates new - MSTR

sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/trading/agency_desk")
from lib import talos_utils

sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility
import utils as aql_utils

os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
google_sheet_key = aql_utils.read_secret(os.environ.get("GOOGLE_SHEET_KEY"))
gsheet_key = json.loads(google_sheet_key)
worksheet_name = "Order Update Script Sheet"
tab_name = "DG_9_20"
gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)

df = gsu._get_current_sheet_df(tab_name, 0)

# Initialize an empty dictionary
orders_list_gs = {}

# Iterate through the DataFrame and populate the orders_list_gs dictionary
for _, row in df.iterrows():
    try:
        orders_list_gs[row['order_id']] = {
            "client_name": row['client_name'],
            "order_desc": row['order_desc'],
            "print_output": bool(row['print_output']),
            "talos_env": row['talos_env']
        }
    except Exception as e:
        print(f"Error processing order_id {row['order_id']}: {e}")

# Start the timer
# startTime = time.time()

mstr_orders_comms_df = calculate_comms(order_details=orders_list_gs, client_name="Microstrategy", commission = 0)
mstr_update = generate_order_update_text(mstr_orders_comms_df, show_finished_orders=True)
print(mstr_update)

/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Error processing order_id 711b2630-195f-4ec7-b3d6-b4848fd126cc: 'talos_env'
Error processing order_id 2c7adc11-cd6c-405a-9a92-9d8a9d8f2dfa: 'talos_env'
Error processing order_id 83c796f4-fbd2-4e8b-af3b-67af74fd0543: 'talos_env'
Error processing order_id 7e231f20-92c2-4476-8200-43d9fc545e46: 'talos_env'
Error processing order_id 1447108d-bf9f-4683-b5f8-431d9ee15529: 'talos_env'
Error processing order_id 536c4673-23a4-4ecd-b89e-75f8531cf832: 'talos_env'
Error processing order_id d75ea9e6-7bf4-422c-8806-9dcb123e6016: 'talos_env'
Error processing order_id 62ebd1dc-2312-44a8-8a02-1e9ffde18980: 'talos_env'


KeyError: 'Strategy'

In [29]:
mstr_orders_comms_df.sum()

ClientName          MicroStrategy IncorporatedMicroStrategy Incorp...
OrderDesc           Order 7- $60,000,000 TWAP with 64,000 limit pr...
trade_side                                                     buybuy
bought_amount                                          1,557.28350275
bought_asset                                                   BTCBTC
sold_amount                                       98,000,000.00000000
sold_asset                                                     USDUSD
commission                                                 1.09086205
comms_ccy                                                      BTCBTC
price_with_comms                                     126,041.79375635
talos_price                                          125,953.56450072
amount_gross                                           1,558.37436480
OrderType                                        SteadyPaceSteadyPace
OrdStatus                                                    DoneDone
Symbol              

In [9]:
###orders updates new - MSTR

sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/trading/agency_desk")
from lib import talos_utils

sys.path.append(os.environ["HOME"] + "/anchorage/source/python/lib/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility # type: ignore
import utils as aql_utils # type: ignore

os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
google_sheet_key = aql_utils.read_secret(os.environ.get("GOOGLE_SHEET_KEY"))
gsheet_key = json.loads(google_sheet_key)
worksheet_name = "Order Update Script Sheet"
tab_name = "pantera"
gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)

df = gsu._get_current_sheet_df(tab_name, 0)

# Initialize an empty dictionary
orders_list_gs = {}

# Iterate through the DataFrame and populate the orders_list_gs dictionary
for _, row in df.iterrows():
    try:
        orders_list_gs[row['order_id']] = {
            "client_name": row['client_name'],
            "order_desc": row['order_desc'],
            "print_output": bool(row['print_output']),
            "talos_env": row['talos_env']
        }
    except Exception as e:
        print(f"Error processing order_id {row['order_id']}: {e}")

# Start the timer
# startTime = time.time()

mstr_orders_comms_df = calculate_comms(order_details=orders_list_gs, client_name="a16z", commission = 10)
mstr_update = generate_order_update_text(mstr_orders_comms_df, show_finished_orders=True)
print(mstr_update)

/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Order Update

Open Orders:

Buy 19,300,000 USD worth of BTC
Ribbit OB1 (Master), L.P. bought 5.05943870 BTC @ $62,330.88631623 for $315,359.30 net

Cumulatively:
Ribbit OB1 (Master), L.P. bought 5.05943870 BTC @ $62,330.88662582 for $315,359.30 net
Ribbit OB1 (Master), L.P. bought 79.81048160 BTC @ 61,688.30268925 for 4,923,373.14671589 net

Filled Orders:

Buy 19,300,000 USD worth of BTC
Ribbit OB1 (Master), L.P. bought 79.81048160 BTC @ 61,688.30269209 for 4,923,373.14694228 net



In [57]:
###orders updates new - MSTR

sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/trading/agency_desk")
from lib import talos_utils

sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility
import utils as aql_utils

os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
google_sheet_key = aql_utils.read_secret(os.environ.get("GOOGLE_SHEET_KEY"))
gsheet_key = json.loads(google_sheet_key)
worksheet_name = "Order Update Script Sheet"
tab_name = "a16z_AR"
gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)

df = gsu._get_current_sheet_df(tab_name, 0)

# Initialize an empty dictionary
orders_list_gs = {}

# Iterate through the DataFrame and populate the orders_list_gs dictionary
for _, row in df.iterrows():
    try:
        orders_list_gs[row['order_id']] = {
            "client_name": row['client_name'],
            "order_desc": row['order_desc'],
            "print_output": bool(row['print_output']),
            "talos_env": row['talos_env']
        }
    except Exception as e:
        print(f"Error processing order_id {row['order_id']}: {e}")

# Start the timer
# startTime = time.time()

mstr_orders_comms_df = calculate_comms(order_details=orders_list_gs, client_name="Microstrategy", commission = 10)
mstr_update = generate_order_update_text(mstr_orders_comms_df, show_finished_orders=True)
print(mstr_update)

/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Order Update

Open Orders:

TWAP with $20 limit price
a16z sold 16,258.20000000 AR @ $20.73648156 for $337,137.86 net



In [16]:
###orders updates new - MSTR

sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/trading/agency_desk")
from lib import talos_utils

sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility
import utils as aql_utils

os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
google_sheet_key = aql_utils.read_secret(os.environ.get("GOOGLE_SHEET_KEY"))
gsheet_key = json.loads(google_sheet_key)
worksheet_name = "Order Update Script Sheet"
tab_name = "DG_9_20"
gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)

df = gsu._get_current_sheet_df(tab_name, 0)

# Initialize an empty dictionary
orders_list_gs = {}

# Iterate through the DataFrame and populate the orders_list_gs dictionary
for _, row in df.iterrows():
    try:
        orders_list_gs[row['order_id']] = {
            "client_name": row['client_name'],
            "order_desc": row['order_desc'],
            "print_output": bool(row['print_output']),
            "talos_env": row['talos_env']
        }
    except Exception as e:
        print(f"Error processing order_id {row['order_id']}: {e}")

# Start the timer
# startTime = time.time()

mstr_orders_comms_df = calculate_comms(order_details=orders_list_gs, client_name="a16z", commission = 0)
mstr_update = generate_order_update_text(mstr_orders_comms_df, show_finished_orders=True)
print(mstr_update)

/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Order Update

Open Orders:

SEI
Distributed Global Ventures III LP sold 31,806.04937000 SEI @ $0.41555138 for $13,217.05 net

WLD
Distributed Global Ventures III LP sold 13,596.23300000 WLD @ 1.60381145 for 21,805.79416227 net

Cumulatively:
Distributed Global Ventures LP sold 727,567.62180000 FLOW @ 0.60045618 for 436,872.47487771 net
Distributed Global Ventures III LP sold 1,528,300.83951000 SEI @ $0.43255401 for $661,072.66 net
Distributed Global Ventures III LP sold 506,615.51300000 WLD @ 1.83811107 for 931,215.58032293 net

Filled Orders:

sell via TWAP to USD 2,466,767.34 FLOW over 14 days with a limit of $0.55
Distributed Global Ventures LP sold 727,567.62180000 FLOW @ 0.60045618 for 436,872.47487771 net

Sell via TWAP to USD 3,472,222.22 SEI over 14 days with a limit of $0.32
Distributed Global Ventures III LP sold 1,496,494.79014000 SEI @ $0.43291538 for $647,855.61 net

Sell via TWAP to USD 1,269,776.75 WLD over 14 days with a limit of $1.60
Distributed Global Ventures III LP